<a href="https://colab.research.google.com/github/ruth-ann/cs399-f21/blob/main/finetune_mbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q transformers

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 65.9 MB/s 
     |████████████████████████████████| 3.3 MB 54.8 MB/s 
     |████████████████████████████████| 895 kB 58.4 MB/s 
     |████████████████████████████████| 56 kB 2.7 MB/s 


In [4]:
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [5]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'


In [6]:
from google.colab import files
uploaded = files.upload()

Saving bzd.tsv to bzd.tsv


In [16]:
import io
raw_df = pd.read_csv('bzd.tsv', delimiter='\t')
# raw_df['list'] = raw_df[raw_df.columns[3:]].values.tolist()
# df = raw_df[['premise', 'hypothesis', 'list']].copy()
raw_df.head()


df = raw_df[['premise', 'hypothesis', 'label']].copy()
df.head()





,premise,hypothesis,label
0,"Ẽnã ie' iche: ãmìx, ye' tso' ù ã.",I ãmìx ichàke irir bét wèxs tabèchka shko ẽ' w...,neutral
1,"Ẽnã ie' iche: ãmìx, ye' tso' ù ã.",Ie' kèq wã ujtè yèqnẽ ét.,contradiction
2,"Ẽnã ie' iche: ãmìx, ye' tso' ù ã.",I ãmìx ã iché irir tö ye' démĩnẽ ù ã.,entailment
3,"Kèq ie' éxn ã ĩàxnã tö ióqk ie' mĩ yéqs, e's t...","Kèq ye' déqrule Washington ã, e' kũéxkĩ míxk y...",neutral
4,"Kèq ie' éxn ã ĩàxnã tö ióqk ie' mĩ yéqs, e's t...",Ye' éxn ã ĩàxnã yêqsyêqs ë tö ì sãũk ye' míxke...,contradiction


In [20]:

train_size = 0.8
train_df=df.sample(frac=train_size,random_state=200)
val_df=df.drop(train_df.index).reset_index(drop=True)
train_df = train_df.reset_index(drop=True)
train_df.head()

,premise,hypothesis,label
0,"Oh, irir Snake River, oh Snake River tkabèq tã...","Éwa tã i ajkà kũéxkĩ, Snake River kèq tkabèq t...",contradiction
1,"Erë, mmm..., e' wa tã be' wòqbatsö chkòq i kũã...",Be' wòqbatsòqmĩ chkòq ulìtãnẽ wa?,entailment
2,"Wòqshàxë, irir skéxl tso' ie, aû kèq irir e' k...",Yö iklóqwẽ tö be' káxwöta e' ã káxwö mũk ĩa àx...,entailment
3,Aû kèq irir abogado èq ãñìxes oficiales policí...,I ulà wa i ulà wa i ajkà téqrke i ujté judicia...,entailment
4,Mĩàx i kíxmẽ kũéxkĩ Museo de Arte e' ã de Indi...,Sa' tsèqnã bua'ë tö Museo kíxmèxnã.,entailment


In [41]:
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pickle
import os
from transformers import BertTokenizer

class AmericasNLIDataMBert(Dataset):


  def __init__(self, train_df, val_df):
    self.label_dict = {'entailment': 0, 'contradiction': 1, 'neutral': 2}

    self.train_df = train_df
    self.val_df = val_df

    self.base_path = '/content/'
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True) 
    self.train_data = None
    self.val_data = None
    self.init_data()

  def init_data(self):
    self.train_data = self.load_data(self.train_df)
    self.val_data = self.load_data(self.val_df)


  def load_data(self, df):
      MAX_LEN = 512
      token_ids = []
      mask_ids = []
      seg_ids = []
      y = []

      premise_list = df['premise'].to_list()
      hypothesis_list = df['hypothesis'].to_list()
      label_list = df['label'].to_list()

      for (premise, hypothesis, label) in zip(premise_list, hypothesis_list, label_list):
        premise_id = self.tokenizer.encode(premise, add_special_tokens = False)
        hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False)
        pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
        premise_len = len(premise_id)
        hypothesis_len = len(hypothesis_id)

        segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
        attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))  # mask padded values

        token_ids.append(torch.tensor(pair_token_ids))
        seg_ids.append(segment_ids)
        mask_ids.append(attention_mask_ids)
        y.append(self.label_dict[label])
      
      token_ids = pad_sequence(token_ids, batch_first=True)
      mask_ids = pad_sequence(mask_ids, batch_first=True)
      seg_ids = pad_sequence(seg_ids, batch_first=True)
      y = torch.tensor(y)
      dataset = TensorDataset(token_ids, mask_ids, seg_ids, y)
      print(len(dataset))
      return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    train_loader = DataLoader(
      self.train_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    val_loader = DataLoader(
      self.val_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    return train_loader, val_loader
americas_nli_dataset = AmericasNLIDataMBert(train_df, val_df)

594
149


In [ ]:

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-multilingual-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 6)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

In [44]:
#@title
MAX_LEN = 200
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

In [49]:
from transformers import BertForSequenceClassification, AdamW


model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", num_labels=3)

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

In [50]:

def multi_acc(y_pred, y_test):
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc

import time

EPOCHS = 5

def train(model, train_loader, val_loader, optimizer):  
  total_step = len(train_loader)

  for epoch in range(EPOCHS):
    start = time.time()
    model.train()
    total_train_loss = 0
    total_train_acc  = 0
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(train_loader):
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)

      loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels).values()

      acc = multi_acc(prediction, labels)

      loss.backward()
      optimizer.step()
      
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)
    model.eval()
    total_val_acc  = 0
    total_val_loss = 0
    with torch.no_grad():
      for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(val_loader):
        optimizer.zero_grad()
        pair_token_ids = pair_token_ids.to(device)
        mask_ids = mask_ids.to(device)
        seg_ids = seg_ids.to(device)
        labels = y.to(device)
        
        loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels).values()
        
        acc = multi_acc(prediction, labels)

        total_val_loss += loss.item()
        total_val_acc  += acc.item()

    val_acc  = total_val_acc/len(val_loader)
    val_loss = total_val_loss/len(val_loader)
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)

    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [51]:
train_loader, val_loader = americas_nli_dataset.get_data_loaders()

In [ ]:
train(model, train_loader, val_loader, optimizer)

Epoch 1: train_loss: 1.1165 train_acc: 0.3275 | val_loss: 1.1203 val_acc: 0.3101
00:12:58.97
Epoch 2: train_loss: 1.1067 train_acc: 0.3189 | val_loss: 1.0967 val_acc: 0.3637
00:12:53.17
Epoch 3: train_loss: 1.1032 train_acc: 0.3602 | val_loss: 1.1151 val_acc: 0.3101
00:12:48.57
Epoch 4: train_loss: 1.1041 train_acc: 0.3490 | val_loss: 1.0983 val_acc: 0.3506
00:12:47.96
